In [1]:
import pandas as pd
from google.cloud import bigquery
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
import string
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow import keras
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
bqclient = bigquery.Client()

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/emma_tebbe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/emma_tebbe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/emma_tebbe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:

query = (
    """
    SELECT * FROM `w266-313317.final_project.tag_asset_count`
    WHERE language = 'en'
    """
)
result = bqclient.query(query).to_dataframe()
result

,tag,language,auto_generated,organization_id,asset_count
0,Dust Extractors,en,False,13770,71
1,TSC 55,en,False,13770,44
2,Social_Media_Posts,en,False,16816,33
3,Triple Teazer,en,False,13887,40
4,new,en,False,14215,62
...,...,...,...,...,...
128651,Gesture,en,True,18161,17
128652,Tan,en,True,18161,17
128653,LineLED 36X2 Imagery,en,False,18161,17
128654,Kilo Imagery,en,False,18161,17


In [11]:
result['rare_tag'] = result['asset_count'] <= 3
result['rare_tag'] = result['rare_tag'].astype(int)

,tag,language,auto_generated,organization_id,asset_count,rare_tag
0,Dust Extractors,en,False,13770,71,0
1,TSC 55,en,False,13770,44,0
2,Social_Media_Posts,en,False,16816,33,0
3,Triple Teazer,en,False,13887,40,0
4,new,en,False,14215,62,0
...,...,...,...,...,...,...
128651,Gesture,en,True,18161,17,0
128652,Tan,en,True,18161,17,0
128653,LineLED 36X2 Imagery,en,False,18161,17,0
128654,Kilo Imagery,en,False,18161,17,0


In [16]:
result['rare_tag'].sum()

82359

In [12]:
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop_words = set(stopwords.words('english'))


In [13]:
result_np = result[['tag','language','auto_generated','organization_id','asset_count']].to_numpy()

In [14]:
def word_preprocessing(word):
    lower = word.lower()
    punct_replacer = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    rem_punct = lower.translate(punct_replacer)
    lemma = [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(rem_punct)]
    rem_stop = [w for w in lemma if not w in stop_words]
    rem_digits = [re.sub('\d', '<dig>', i) for i in rem_stop]
    return rem_digits, word
lemmatized = [word_preprocessing(i[0]) for i in result_np]
lemma_df = pd.DataFrame(lemmatized, columns = ['lemmatized','tag']).set_index('tag')

In [15]:

query = (
    """
    SELECT tag, cast(asset_id as string) as asset_id, auto_generated, cast(organization_id as string) as organization_id, source
    FROM `w266-313317.final_project.raw_tags`
    WHERE language = 'en'
    """
)
full_tags = bqclient.query(query).to_dataframe()

tag                object
asset_id           object
auto_generated       bool
organization_id    object
source             object
dtype: object

In [ ]:
full_df = full_tags.join(lemma_df, on = 'tag')
full_df

In [1]:
join_df = lemma_df.join(result[['tag', 'rare_tag']].set_index('tag'), on='tag')

NameError: name 'lemma_df' is not defined